In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [23]:
 #sklearn preproc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import make_scorer

from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor

In [24]:
# Vacination Dataset
url_vaccination = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv'
url_ages = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations-by-age-group.csv'

# Index URLs
url_index_strigency = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/stringency_index_avg.csv'
url_index_gov_response = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/government_response_index_avg.csv'
url_index_health = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/containment_health_index_avg.csv'
url_index_economic = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/economic_support_index.csv'

# Indicators URLs
# C sub-indicators
url_c1 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c1m_school_closing.csv'
url_c2 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c2m_workplace_closing.csv'
url_c3 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c3m_cancel_public_events.csv'
url_c4 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c4m_restrictions_on_gatherings.csv'
url_c5 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c5m_close_public_transport.csv'
url_c6 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c6m_stay_at_home_requirements.csv'
url_c7 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c7m_movementrestrictions.csv'
url_c8 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c8ev_internationaltravel.csv'

# E sub-indicators
url_e1 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/e1_income_support.csv'
url_e2 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/e2_debtrelief.csv'

# H sub-indicators
url_h1 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/h1_public_information_campaigns.csv'
url_h2 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/h2_testing_policy.csv'
url_h3 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/h3_contact_tracing.csv'
url_h6 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/h6m_facial_coverings.csv'
url_h7 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/h7_vaccination_policy.csv'
url_h8 = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/h8m_protection_of_elderly_ppl.csv'

# Target URLs
url_cases = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/confirmed_cases.csv'
url_deaths = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/confirmed_deaths.csv'


In [25]:
# DataFrame Index
df_strigency_raw = pd.read_csv(url_index_strigency)
df_gov_response_raw = pd.read_csv(url_index_gov_response)
df_health_raw = pd.read_csv(url_index_health)
df_economic_raw = pd.read_csv(url_index_economic)

# DataFrame Vaccination
df_vaccination_raw = pd.read_csv(url_vaccination)
df_ages_raw = pd.read_csv(url_ages)

df_c1_raw = pd.read_csv(url_c1)
df_c2_raw = pd.read_csv(url_c2)
df_c3_raw = pd.read_csv(url_c3)
df_c4_raw = pd.read_csv(url_c4)
df_c5_raw = pd.read_csv(url_c5)
df_c6_raw = pd.read_csv(url_c6)
df_c7_raw = pd.read_csv(url_c7)
df_c8_raw = pd.read_csv(url_c8)

df_e1_raw = pd.read_csv(url_e1)
df_e2_raw = pd.read_csv(url_e2)

df_h1_raw = pd.read_csv(url_h1)
df_h2_raw = pd.read_csv(url_h2)
df_h3_raw = pd.read_csv(url_h3)
df_h6_raw = pd.read_csv(url_h6)
df_h7_raw = pd.read_csv(url_h7)
df_h8_raw = pd.read_csv(url_h8)


# Data Frame target
df_cases_raw = pd.read_csv(url_cases)
df_deaths_raw = pd.read_csv(url_deaths)

In [26]:
def data_cleaning_all_index(name_data_table):
    trans_table=name_data_table.groupby('country_code').sum().T.drop('Unnamed: 0')
    trans_table.index = pd.to_datetime(trans_table.index)
    return trans_table
    


In [27]:
def data_cleaning_all_indicator(name_data_table):
    trans_table=name_data_table.groupby('country_code').mean().round(decimals = 0).T.drop('Unnamed: 0')
    trans_table.index = pd.to_datetime( trans_table.index)
    return trans_table

In [39]:
df_strigency = data_cleaning_all_index(df_strigency_raw)
df_gov_response = data_cleaning_all_index(df_gov_response_raw)
df_health= data_cleaning_all_index(df_health_raw)
df_economic= data_cleaning_all_index(df_economic_raw)


# DataFrames Indicator
df_c1 = data_cleaning_all_indicator(df_c1_raw)
df_c2 = data_cleaning_all_indicator(df_c2_raw)
df_c3 = data_cleaning_all_indicator(df_c3_raw)
df_c4 = data_cleaning_all_indicator(df_c4_raw)
df_c5 = data_cleaning_all_indicator(df_c5_raw)
df_c6 = data_cleaning_all_indicator(df_c6_raw)
df_c7 = data_cleaning_all_indicator(df_c6_raw)
df_c8 = data_cleaning_all_indicator(df_c7_raw)

df_e1 =data_cleaning_all_indicator(df_e1_raw)
df_e2 = data_cleaning_all_indicator(df_e2_raw)

df_h1 = data_cleaning_all_indicator(df_h1_raw)
df_h2 = data_cleaning_all_indicator(df_h2_raw)
df_h3 = data_cleaning_all_indicator(df_h3_raw)
df_h6 = data_cleaning_all_indicator(df_h6_raw)
df_h7 = data_cleaning_all_indicator(df_h7_raw)
df_h8 =data_cleaning_all_indicator(df_h8_raw)


# Data Frame target
df_cases =data_cleaning_all_index(df_cases_raw)
df_deaths = data_cleaning_all_index(df_deaths_raw)

# Cleaning Vaccination Features
df_vaccination = df_vaccination_raw.copy()
df_vaccination = df_vaccination[['date','iso_code','people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred']]
df_vaccination=df_vaccination.set_index('date')
#df_vaccination = pd.to_datetime( df_vaccination.index)
df_vaccination=df_vaccination[df_vaccination['iso_code']=='USA']
df_vaccination=df_vaccination[['people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred']]
#df_vaccination.rename(columns = {'iso_code': 'country_code'}, inplace = True)
df_vaccination.index = pd.to_datetime( df_vaccination.index)
df_vaccination.head()

,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
date,,,
2021-09-30,64.80,57.32,1.89
2021-10-01,64.90,57.43,2.09
2021-10-02,64.95,57.50,2.17
2021-10-03,64.98,57.54,2.21
2021-10-04,65.05,57.61,2.36


In [40]:
country = 'USA'

def country_output(country):
    df_gov_response_usa=df_gov_response['USA']
    country_index=df_gov_response_usa
    country_index=pd.DataFrame(country_index)
    country_index.columns = ['gov_response']
    country_index.insert(0, 'containment_and_health', df_health[country])
    country_index.insert(1, 'stringency', df_strigency[country])
    country_index.insert(2,'economics_sup',df_economic[country])
    country_index.insert(3,'total_cases',df_cases[country])
    country_index.insert(4,'new_cases',df_cases[country]-df_cases[country].shift(1))
    country_index.insert(5,'total_deaths',df_deaths[country])
    country_index.insert(6,'new_deaths',df_deaths[country] - df_deaths[country].shift(1))
    country_index.index.name='date'
    
    #indicator
    df_c2_usa=df_c2[country]
    country_indicator= df_c2_usa
    country_indicator=pd.DataFrame(country_indicator)
    country_indicator.columns = ['workplace_closing']
    country_indicator.insert(0, 'cancel_public_events', df_c3[country])
    country_indicator.insert(1, 'restrictions_on_gathering', df_c4[country])
    country_indicator.insert(2,'close_public_transport',df_c5[country])
    country_indicator.insert(3,'stay_at_home_requirements',df_c6[country])
    country_indicator.insert(4,'restrictions_on_internal_movement',df_c7[country])
    country_indicator.insert(5,'international_travel_controls',df_c8[country])
    country_indicator.insert(6,'income_support',df_e1[country])
    country_indicator.insert(7,'debt/contract_relief',df_e2[country])
    country_indicator.insert(8,'public_information_campaigns',df_h1[country])
    country_indicator.insert(9,'testing_policy',df_h2[country])
    country_indicator.insert(10,'contact_tracing',df_h3[country])
    country_indicator.insert(11,'facial_coverings',df_h6[country])
    country_indicator.insert(12,'vaccination_policy',df_h7[country])
    country_indicator.insert(13,'protection_of_elderly_people',df_h8[country])
    country_indicator.insert(14,'total_cases',df_cases[country])
    country_indicator.insert(15,'new_cases',df_cases[country]-df_cases[country].shift(1))
    country_indicator.insert(16,'total_deaths',df_deaths[country])
    country_indicator.insert(17,'new_deaths',df_deaths[country] - df_deaths[country].shift(1))
    country_indicator.index.name='date'
    
    country_vaccination=df_vaccination
    country_index = country_index.merge(country_vaccination, how = 'left' , on = 'date')
    country_indicator=country_indicator.merge(country_vaccination, how = 'left' , on = 'date')
 
    return country_index,country_indicator
    
    

In [42]:
country_index_usa=country_output(country)[0]

In [43]:
country_index_usa

,containment_and_health,stringency,economics_sup,total_cases,new_cases,total_deaths,new_deaths,gov_response,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
date,,,,,,,,,,,
2020-01-01,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.00,NaN,NaN,NaN
2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,NaN,NaN,NaN
2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,NaN,NaN,NaN
2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,NaN,NaN,NaN
2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-04,0.0,0.0,0.0,188464761.0,10524.0,2089425.0,1.0,2146.68,79.24,67.58,39.5
2022-09-05,0.0,0.0,0.0,188503831.0,39070.0,2089567.0,142.0,2146.68,79.25,67.58,39.5
2022-09-06,0.0,0.0,0.0,188761844.0,258013.0,2090809.0,1242.0,2146.68,79.25,67.58,39.5


In [44]:
country_indicator_usa=country_output(country)[1]

In [45]:
country_indicator_usa

,cancel_public_events,restrictions_on_gathering,close_public_transport,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt/contract_relief,public_information_campaigns,testing_policy,...,vaccination_policy,protection_of_elderly_people,total_cases,new_cases,total_deaths,new_deaths,workplace_closing,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN
2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-04,NaN,NaN,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,188464761.0,10524.0,2089425.0,1.0,1.0,79.24,67.58,39.5
2022-09-05,NaN,NaN,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,188503831.0,39070.0,2089567.0,142.0,1.0,79.25,67.58,39.5
2022-09-06,NaN,NaN,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,188761844.0,258013.0,2090809.0,1242.0,1.0,79.25,67.58,39.5


In [46]:
country_index_usa_csv=country_index_usa.to_csv()

In [19]:
#### CSV Data out project directory
csv_dir = '../data/out_csv/'

In [20]:
compression_opts = dict(method='zip',
                        archive_name='usa_index.csv')  
country_index_usa.to_csv(csv_dir + 'index_usa_csv.zip',
          compression=compression_opts) 

In [21]:
compression_opts = dict(method='zip',
                        archive_name='usa_indicator.csv')  
country_indicator_usa.to_csv(csv_dir + 'indicator_usa_csv.zip',
          compression=compression_opts) 